# **SETUP**

## Import

In [1]:
from __future__ import division
import pickle
import numpy as np
import os
from openrec.tf1.legacy import ImplicitModelTrainer
from openrec.tf1.legacy.utils import ImplicitDataset
from openrec.tf1.legacy.utils.evaluators import ImplicitEvalManager
from openrec.tf1.legacy.recommenders import CML, BPR
from openrec.tf1.legacy.utils.evaluators import AUC
from openrec.tf1.legacy.utils.samplers import PairwiseSampler

## Init

In [2]:
auc_results = []
recall_results = []

seed = 76424236
np.random.seed(seed=seed)

folder_name = f"./Dataset/"

if os.path.exists(folder_name) == False:
    os.makedirs(folder_name)

## Load training set

In [3]:
raw_data = dict()
raw_data['train_data'] = np.load(folder_name + "training_arr.npy")
raw_data['test_data_pos'] = np.load(folder_name + "biased-test_arr_pos.npy")
raw_data['test_data_neg'] = np.load(folder_name + "biased-test_arr_neg.npy")
raw_data['max_user'] = 15401
raw_data['max_item'] = 1001
batch_size = 8000
test_batch_size = 1000
display_itr = 1000

train_dataset = ImplicitDataset(raw_data['train_data'], raw_data['max_user'], raw_data['max_item'], name='Train')
test_dataset_pos = ImplicitDataset(raw_data['test_data_pos'], raw_data['max_user'], raw_data['max_item'])
test_dataset_neg = ImplicitDataset(raw_data['test_data_neg'], raw_data['max_user'], raw_data['max_item'])

## Define Model

In [4]:
#Code to avoid tf using cached embeddings
import tensorflow as tf
tf.compat.v1.reset_default_graph()

cml_model = CML(batch_size=batch_size, max_user=train_dataset.max_user(), max_item=train_dataset.max_item(),
    dim_embed=50, l2_reg=0.001, opt='Adam', sess_config=None)
sampler = PairwiseSampler(batch_size=batch_size, dataset=train_dataset, num_process=4)
model_trainer = ImplicitModelTrainer(batch_size=batch_size, test_batch_size=test_batch_size,
                                     train_dataset=train_dataset, model=cml_model, sampler=sampler,
                                     eval_save_prefix=folder_name+"yahoo",
                                     item_serving_size=500)
auc_evaluator = AUC()

cml_model.load(folder_name+"cml-yahoo")


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor




Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






2024-02-16 16:10:50.852517: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2024-02-16 16:10:50.871522: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fbd79d820b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-02-16 16:10:50.871536: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


INFO:tensorflow:Restoring parameters from ./Dataset/cml-yahoo


# **DEFINE FUNCTION**

In [5]:
def eq(infilename, infilename_neg, trainfilename, gamma=-1.0, K=30):
    infile = open(infilename, 'rb')
    infile_neg = open(infilename_neg, 'rb')
    P = pickle.load(infile)
    infile.close()
    P_neg = pickle.load(infile_neg)
    infile_neg.close()
    NUM_NEGATIVES = P["num_negatives"]
    #
    for theuser in P["users"]:
        neg_items = list(P_neg["user_items"][theuser][NUM_NEGATIVES:])
        neg_scores = list(P_neg["results"][theuser][NUM_NEGATIVES:])
        P["user_items"][theuser] = list(neg_items) + list(P["user_items"][theuser][NUM_NEGATIVES:])
        P["results"][theuser] = list(neg_scores) + list(P["results"][theuser][NUM_NEGATIVES:])
    #
    Zui = dict()
    Ni = dict()
    # fill in dictionary Ni
    trainset = np.load(trainfilename)
    for i in trainset['item_id']:
        if i in Ni:
            Ni[i] += 1
        else:
            Ni[i] = 1
    del trainset
    # count #users with non-zero item frequencies
    nonzero_user_count = 0
    for theuser in P["users"]:
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        for pos_item in pos_items:
            if pos_item in Ni:
                nonzero_user_count += 1
                break
    # fill in dictionary Zui
    for theuser in P["users"]:
        all_scores = np.array(P["results"][theuser])
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        pos_scores = P["results"][theuser][len(P_neg["results"][theuser][NUM_NEGATIVES:]):]
        for i, pos_item in enumerate(pos_items):
            pos_score = pos_scores[i]
            Zui[(theuser, pos_item)] = float(np.sum(all_scores > pos_score))
    # calculate per-user scores
    sum_user_auc = 0.0
    sum_user_recall = 0.0
    for theuser in P["users"]:
        numerator_auc = 0.0
        numerator_recall = 0.0
        denominator = 0.0
        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            if theitem not in Ni:
                continue
            pui = np.power(Ni[theitem], (gamma + 1) / 2.0)
            numerator_auc += (1 - Zui[(theuser, theitem)] / len(P["user_items"][theuser])) / pui
            # Calcolo il Recall a 30, vedi nota 6 paper
            if Zui[(theuser, theitem)] < K:
                numerator_recall += 1.0 / pui
            denominator += 1 / pui
        if denominator > 0:
            sum_user_auc += numerator_auc / denominator
            sum_user_recall += numerator_recall / denominator

    return {
        "auc"       : sum_user_auc / nonzero_user_count,
        "recall"    : sum_user_recall / nonzero_user_count
    }

# **TEST**

## Generate Raw Results

In [6]:
model_trainer._eval_manager = ImplicitEvalManager(evaluators=[auc_evaluator])
model_trainer._num_negatives = 200
model_trainer._exclude_positives([train_dataset, test_dataset_pos, test_dataset_neg])
model_trainer._sample_negatives(seed=10)

model_trainer._eval_save_prefix = folder_name+"cml-yahoo-test-pos-biased"
model_trainer._evaluate_partial(test_dataset_pos)

model_trainer._eval_save_prefix = folder_name+"cml-yahoo-test-neg-biased"
model_trainer._evaluate_partial(test_dataset_neg)

[Subsampling negative items]


100%|██████████| 2296/2296 [01:01<00:00, 37.14it/s]


{'AUC': [0.4911655405405405,
  0.5114309764309763,
  0.47823232323232323,
  0.47508333333333325,
  0.49118644067796613,
  0.4970508474576271,
  0.5332881355932203,
  0.5419360269360269,
  0.5159121621621622,
  0.4557457627118644,
  0.4485690235690237,
  0.505051724137931,
  0.5000242718446601,
  0.5126166666666667,
  0.4587068965517241,
  0.49633333333333324,
  0.5013545150501673,
  0.5601890034364261,
  0.5084113712374583,
  0.5018791946308725,
  0.4925418060200669,
  0.5108644067796609,
  0.4572986577181207,
  0.4853703703703703,
  0.5253716216216217,
  0.4938144329896907,
  0.47029209621993123,
  0.5371237458193979,
  0.5079180887372015,
  0.5237710437710438,
  0.5224161073825504,
  0.5221666666666668,
  0.5144655172413792,
  0.5005183946488295,
  0.48801003344481614,
  0.49296296296296305,
  0.5151833333333333,
  0.5011904761904763,
  0.5173244147157191,
  0.5141750841750841,
  0.5172727272727272,
  0.4631358885017422,
  0.4817457627118644,
  0.5158053691275167,
  0.473729096989966

## Compute results

In [7]:
eq(folder_name+"cml-yahoo-test-pos-biased_evaluate_partial.pickle", folder_name+"cml-yahoo-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=1.5)

{'auc': 0.772736165138026, 'recall': 0.45398828104948435}

In [8]:
eq(folder_name+"cml-yahoo-test-pos-biased_evaluate_partial.pickle", folder_name+"cml-yahoo-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=2)

{'auc': 0.7651120425208878, 'recall': 0.4394679767082}

In [9]:
eq(folder_name+"cml-yahoo-test-pos-biased_evaluate_partial.pickle", folder_name+"cml-yahoo-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=3)

{'auc': 0.7550797295856848, 'recall': 0.42104252841483814}

In [10]:
#gamma 1.5
{'auc': 0.8449884377871792-0.6841237836533649, 'recall': 0.5929405866566257-0.24769933731230948}

{'auc': 0.16086465413381434, 'recall': 0.34524124934431627}

In [11]:
#Gamma 2
{'auc': 0.8409330479463847-0.6805190093975312, 'recall': 0.5829036824161936-0.24407427229889453}

{'auc': 0.1604140385488535, 'recall': 0.33882941011729906}

In [12]:
#Gamma 3
{'auc': 0.8357291786502742-0.6758300383034919, 'recall': 0.5702390779177972-0.239510164195683}


{'auc': 0.15989914034678232, 'recall': 0.33072891372211416}

In [20]:
def aoa(infilename, infilename_neg, trainfilename, K=30):
    infile = open(infilename, 'rb')
    infile_neg = open(infilename_neg, 'rb')
    P = pickle.load(infile)
    infile.close()
    P_neg = pickle.load(infile_neg)
    infile_neg.close()
    NUM_NEGATIVES = P["num_negatives"]
    #
    for theuser in P["users"]:
        neg_items = list(P_neg["user_items"][theuser][NUM_NEGATIVES:])
        neg_scores = list(P_neg["results"][theuser][NUM_NEGATIVES:])
        P["user_items"][theuser] = list(neg_items) + list(P["user_items"][theuser][NUM_NEGATIVES:])
        P["results"][theuser] = list(neg_scores) + list(P["results"][theuser][NUM_NEGATIVES:])
    #
    Zui = dict()
    Ni = dict()
    # fill in dictionary Ni
    trainset = np.load(trainfilename)
    for i in trainset['item_id']:
        if i in Ni:
            Ni[i] += 1
        else:
            Ni[i] = 1
    del trainset
    # count #users with non-zero item frequencies
    nonzero_user_count = 0
    for theuser in P["users"]:
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        for pos_item in pos_items:
            if pos_item in Ni:
                nonzero_user_count += 1
                break

    # fill in dictionary Zui
    for theuser in P["users"]:
        all_scores = np.array(P["results"][theuser])
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        pos_scores = P["results"][theuser][len(P_neg["results"][theuser][NUM_NEGATIVES:]):]
        for i, pos_item in enumerate(pos_items):
            pos_score = pos_scores[i]
            Zui[(theuser, pos_item)] = float(np.sum(all_scores > pos_score))
    # calculate per-user scores
    sum_user_auc = 0.0
    sum_user_recall = 0.0

    counter = 0

    for theuser in P["users"]:
        numerator_auc = 0.0
        numerator_recall = 0.0
        denominator = 0.0
        #CHECK THIS IS ACTUALLY CORRECT
        #denominator = len(raw_data['train_data'][raw_data['train_data']['user_id'] == theuser]['item_id'])

        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            if theitem not in Ni:
                continue
            numerator_auc += (1 - Zui[(theuser, theitem)] / len(P["user_items"][theuser]))
            # Calcolo il Recall a 30, vedi nota 6 paper
            if Zui[(theuser, theitem)] < K:
                numerator_recall += 1.0
            denominator += 1 

        if denominator > 0:
            counter+=1
            
            sum_user_auc += numerator_auc / denominator
            sum_user_recall += numerator_recall / denominator

    return {
        "auc"       : sum_user_auc / nonzero_user_count,
        "recall"    : sum_user_recall / nonzero_user_count
    }

In [21]:
aoa(folder_name+"cml-yahoo-test-pos-biased_evaluate_partial.pickle", folder_name+"cml-yahoo-test-neg-biased_evaluate_partial.pickle", folder_name+"training_arr.npy")

{'auc': 0.857080935955807, 'recall': 0.6323394943425948}